In [1]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from time import time
import mysql.connector
from IPython.display import display # Allows the use of display() for DataFrames

# Pretty display for notebooks
%matplotlib inline

# Load the cars dataset from csv exported from excell
data = pd.read_csv("Car_DataSet.CSV", delimiter=";")

# Success - Display the first 5 records, as an example
display(data.head(n=5))

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,Dealer
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0,David
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0,NaN
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0,Henry
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0,NaN
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0,Henry


In [2]:
#data cleaning

#DC.1 - mileage: - 
#DC.1 - mileage: -  we keep it in kmpl measure.

#DC.1 - mileage: - we want to remove the kmpl string from the field and keep the float part only
data['mileage'] = data['mileage'].apply(lambda x: str(x).replace('kmpl', ''))

#DC.1 - mileage: -  we want to convert km/kg to kmpl: f=number*1.04.
#DC.1 - mileage: -  moreover, we remove the km/kg string to keep the float part only
data['mileage'] = data['mileage'].apply(lambda x: str(float(str(x).replace('km/kg', ''))*1.40) if "km/kg" in x else x)

#DC.1 - mileage: -  let's convert the whole column to float number
data['mileage'] = data['mileage'].apply(lambda x: float(x) if not pd.isnull(x) else x)

#DC.1 - mileage: -  let's set NaN to empty values
data['mileage'] = data['mileage'].apply(lambda x: np.nan if x == '' or str(x).strip() == '' else x)

# Success - Display the first 5 records, as an example
display(data.head(n=5))

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,Dealer
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248 CC,74 bhp,190Nm@ 2000rpm,5.0,David
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0,NaN
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0,Henry
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0,NaN
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0,Henry


In [3]:
#DC.2 - engine power: - 
#DC.2 - engine power: -  we keep it in CC measure.

#2 - engine power: - we want to remove the CC string from the field and keep the int part only
data['engine'] = data['engine'].apply(lambda x: str(x).replace('CC', ''))


#DC.2 - engine power: -  let's convert the whole column to int number.
#  - engine power: -  Double conversion needed to float first and then int afterwards because of NaN values
data['engine'] = data['engine'].apply(lambda x: float(x) if not pd.isnull(x) and not x.strip() == '' else x)
data['engine'] = data['engine'].apply(lambda x: int(x) if not pd.isnull(x) else x)

#DC.1 - engine power:: -  Let's set NaN to empty values
data['engine'] = data['engine'].apply(lambda x: np.nan if x == '' or str(x).strip() == '' else x)

# Success - Display the first 5 records, as an example
display(data.head(n=5))

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,Dealer
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74 bhp,190Nm@ 2000rpm,5.0,David
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52 bhp,250Nm@ 1500-2500rpm,5.0,NaN
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0,Henry
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90 bhp,22.4 kgm at 1750-2750rpm,5.0,NaN
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0,Henry


In [4]:
#DC.3 - engine max power: - 
#DC.3 - engine max power: -  we keep it in bhp measure.

#DC.3 - engine max power: - we want to remove the bhp string from the field and keep the int part only
data['max_power'] = data['max_power'].apply(lambda x: str(x).replace('bhp', ''))


#DC.3 - engine max power: -  let's convert the whole column to float number.
data['max_power'] = data['max_power'].apply(lambda x: float(x) if not pd.isnull(x) and not x.strip() == '' else x)

#DC.1 - engine max power:: -  Let's set NaN to empty values
data['max_power'] = data['max_power'].apply(lambda x: np.nan if x == '' or str(x).strip() == '' else x)

# Success - Display the first 5 records, as an example
display(data.head(n=5))

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,Dealer
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74.00,190Nm@ 2000rpm,5.0,David
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,250Nm@ 1500-2500rpm,5.0,NaN
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78.00,"12.7@ 2,700(kgm@ rpm)",5.0,Henry
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90.00,22.4 kgm at 1750-2750rpm,5.0,NaN
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.20,"11.5@ 4,500(kgm@ rpm)",5.0,Henry


In [5]:
#DC.4 - engine torque: - 
#DC.4 - engine torque: -  we split it into 3 columns.
#                         First, ToqueNm in Nm measure
#                         Second, TorqueRpmMin in Rpm measure
#                         Third, TorqueRpmMax in Rpm measure


#DC.4 - engine torque: - At first, we replace the ',' text with '.' char, to uniform the standard float numbers fromat
data['torque'] = data['torque'].apply(lambda x: str(x).replace(',', '.'))

#DC.4 - engine torque: - Then, we replace the '~' text with '-' char, to uniform the standard range char separator
data['torque'] = data['torque'].apply(lambda x: str(x).replace('~', '-'))

#DC.4 - engine torque: - Then, we replace the '+/-' text with '-' char, to uniform the standard range char separator
data['torque'] = data['torque'].apply(lambda x: str(x).replace('+/-', ''))

#DC.4 - engine torque: - Then, we replace the '/' text with '@' char, to uniform the standard separator 
data['torque'] = data['torque'].apply(lambda x: str(x).replace('/', '@'))

#DC.4 - engine torque: - Then, we replace the 'at' text with '@' char, to uniform the standard char separator
data['torque'] = data['torque'].apply(lambda x: str(x).replace('at', '@'))


# data is made up of two values,
#                               the first on in Kgm or Nm. We choose the Nm as final measure
#                               the second is a single value or a range, always in Rpm measure

#the two values are always split by an '@', at this point.
#the second value/range is composed by a number or by two numbers split by a '-', at this point

#we want to keep only the numbers without measure string, putting them into the three new columns.

#DC.4 - engine torque: - Let's evaluate deeper the cases and create the three new columns
torque_Nm= []
torque_RpmMin= []
torque_RpmMax= []

#for each string in the column torque
for s in data.torque:
    s = s.strip() #let's trim the string
    #if the string is null, let's put NaN value into the columns
    if pd.isnull(s) or s == '':
        torque_Nm.append(np.nan)
        torque_RpmMin.append(np.nan)
        torque_RpmMax.append(np.nan)
    else:
        #let's see if there is a conversion to be made of the first value
        isKgmToBeConverted = 'kgm' in s.lower() 
        #let's see if it is a case in which there's only the first value in Nm
        isNmOnly = '@' not in s
   
        # then, let's remove usless string parts (unit of measure, brackets, ...)
        new_string = ''
        for c in s:
            if c.isdigit() or c == '.' or c == '@' or c == '-':
                new_string += c
        #let's trim the string
        new_string.strip()
        if pd.isnull(new_string) or new_string == '':
            torque_Nm.append(np.nan)
            torque_RpmMin.append(np.nan)
            torque_RpmMax.append(np.nan)
        else:
            #let's remove the final @ in the cases the original string has 2 @ rather than one: example '1.5@ 4,500(kgm@ rpm)'
            if new_string.endswith('@'):
                new_string = new_string[:-1]
                
           #at this point all strings are in the format: value1@value2_1-value2_2 or value1 if isNmOnly
            parts = new_string.split('@') 

            #let's fill torque_Nm (value1)
            if isKgmToBeConverted:
                torque_Nm.append(float(parts[0])*9.80665)
            else:
                torque_Nm.append(parts[0])

            #let's fill TorqueRpm (value2)
            if isNmOnly:
                torque_RpmMin.append(np.nan)
                torque_RpmMax.append(np.nan)
            else:
                parts_two = parts[1].split('-')
                #let's fill TorqueRpmMin
                torque_RpmMin.append(parts_two[0])
                #let's fill TorqueRpmMax
                if len(parts_two) > 1:
                    torque_RpmMax.append(parts_two[1])
                else:
                    torque_RpmMax.append(np.nan)

#let's remove the old column from the dataset and let's put the 3 new ones we have created
data = data.drop(columns=['torque'])
data['ToqueNm'] = torque_Nm
data['TorqueRpmMin'] = torque_RpmMin
data['TorqueRpmMax'] = torque_RpmMax

# Success - Display the first 5 records, as an example
display(data.head(n=5))


,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,seats,Dealer,ToqueNm,TorqueRpmMin,TorqueRpmMax
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248.0,74.00,5.0,David,190,2000,NaN
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498.0,103.52,5.0,NaN,250,1500,2500
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497.0,78.00,5.0,Henry,124.544455,2.700,NaN
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396.0,90.00,5.0,NaN,219.66896,1750,2750
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298.0,88.20,5.0,Henry,112.776475,4.500,NaN


In [6]:
#data now are cleaned and transformed!
#let's have a look at how many null values we have
print(data.isnull().sum())

name                0
year                0
selling_price       0
km_driven           0
fuel                0
seller_type         0
transmission        0
owner               0
mileage           221
engine            221
max_power         216
seats             221
Dealer           6842
ToqueNm           222
TorqueRpmMin      256
TorqueRpmMax     5439
dtype: int64


In [7]:
#now let's think of filling our DataBase with these data!

#There some entirties in database, that are "enums" of possible values:
#        TransmissionTypes
#        FuelTypes
#        OwnerTypes
#        SellerTypes

#let's find the possible values of these enums.
enums = {'TransmissionTypes': data['transmission'].unique(),
         'FuelTypes': data['fuel'].unique(),
         'OwnerTypes': data['owner'].unique(),
         'SellerTypes': data['seller_type'].unique()
        }

print(enums)

{'TransmissionTypes': array(['Manual', 'Automatic'], dtype=object), 'FuelTypes': array(['Diesel', 'Petrol', 'LPG', 'CNG'], dtype=object), 'OwnerTypes': array(['First Owner', 'Second Owner', 'Third Owner',
       'Fourth & Above Owner', 'Test Drive Car'], dtype=object), 'SellerTypes': array(['Individual', 'Dealer', 'Trustmark Dealer'], dtype=object)}


In [8]:

def insertDBEnum(tablename, enum):
    
    #let's connect to the DB.
    mydb = mysql.connector.connect(
                                       host="localhost",
                                       user="root",
                                       password="root",
                                       database="carssystem"
    )

    mycursor = mydb.cursor()

    #let's create a structure to keep the Ids of the records created
    DBenum = {}

    try:
        #for each element in the enum
        for value in enums[tablename]:
            #search if it already exists on DB
            sql = "SELECT Id "
            sql +=f"FROM {tablename} "
            sql +=f"WHERE Description = '{value}' "
            mycursor.execute(sql)
            myresult = mycursor.fetchone()
            
            #id it doesn't exist
            if myresult is None:
                #we insert it on DB
                sql = f"INSERT INTO {tablename} (Description) VALUES ('{value}');"
                mycursor.execute(sql)
                DBenum[value] = mycursor.lastrowid
                mydb.commit() #let's commit changes
                print(f"Inserted record into {tablename} with id {mycursor.lastrowid} and value {value}")
            else: #if the record already exists
                #we simply track its Id
                DBenum[value] = myresult[0]
                print(f"found record from {tablename} with id {myresult[0]} and value {value}")


    except ValueError: #in case of error, let's print it
        print(ValueError)
    
    #let's close our DB connection
    mycursor.close()
    mydb.close()
        
    print("-------------------")
    print(DBenum)
    print("-------------------")
    return DBenum

In [9]:
DBenums = {}

DBenums['TransmissionTypes'] = insertDBEnum('TransmissionTypes', enums['TransmissionTypes'])
DBenums['FuelTypes'] = insertDBEnum('FuelTypes', enums['FuelTypes'])
DBenums['OwnerTypes'] = insertDBEnum('OwnerTypes', enums['OwnerTypes'])
DBenums['SellerTypes'] = insertDBEnum('SellerTypes', enums['SellerTypes'])

print("-------------------")
print("DB ENUMS--------")
print(DBenums)
print("-------------------")


Inserted record into TransmissionTypes with id 3 and value Manual
Inserted record into TransmissionTypes with id 4 and value Automatic
-------------------
{'Manual': 3, 'Automatic': 4}
-------------------
Inserted record into FuelTypes with id 5 and value Diesel
Inserted record into FuelTypes with id 6 and value Petrol
Inserted record into FuelTypes with id 7 and value LPG
Inserted record into FuelTypes with id 8 and value CNG
-------------------
{'Diesel': 5, 'Petrol': 6, 'LPG': 7, 'CNG': 8}
-------------------
Inserted record into OwnerTypes with id 6 and value First Owner
Inserted record into OwnerTypes with id 7 and value Second Owner
Inserted record into OwnerTypes with id 8 and value Third Owner
Inserted record into OwnerTypes with id 9 and value Fourth & Above Owner
Inserted record into OwnerTypes with id 10 and value Test Drive Car
-------------------
{'First Owner': 6, 'Second Owner': 7, 'Third Owner': 8, 'Fourth & Above Owner': 9, 'Test Drive Car': 10}
-------------------
Ins

In [10]:
#There are some entirties in database, that we know as initial data: the dealers.
#let's have a look at which dealers we have into our data
print(data['Dealer'].unique())


['David' nan 'Henry' 'Anny']


In [11]:
#they're exactly the 3 dealers we know their features of.
#let's addd them.

In [12]:

def insertDealer(Appellation, Name, Surname):
    
    #let's connect to the DB.
    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="root",
      database="carssystem"
    )

    mycursor = mydb.cursor()

    #let's create a structure to keep the Ids of the records created
    Id = -1

    try:
        
        #search if it already exists on DB
        sql = "SELECT Id "
        sql +=f"FROM dealers "
        sql +=f"WHERE Name = '{Name}' "
        mycursor.execute(sql)
        myresult = mycursor.fetchone()
        #id it doesn't exist
        if myresult is None:
            #we insert it on DB
            sql = f"INSERT INTO dealers (Appellation, Name, Surname) "
            sql += " VALUES "
            sql += f" ('{Appellation}','{Name}','{Surname}');"
            mycursor.execute(sql)
            Id = mycursor.lastrowid
            mydb.commit() #let's commit changes
            print(f"Inserted record into dealers with id {mycursor.lastrowid} and Name {Name}")
        else: #if the record already exists
            #we simply track its Id
            Id = myresult[0]
            print(f"found record from dealers with id {myresult[0]} and name {Name}")

    except ValueError: #in case of error, let's print it
        print(ValueError)
    
    #let's close our DB connection
    mycursor.close()
    mydb.close()
        
    return Id


In [13]:
DBdealers = {}
DBdealers['Henry'] = insertDealer('Mr.','Henry','Spelman')
DBdealers['Anny'] = insertDealer('Ms.','Anny','Grouper')
DBdealers['David'] = insertDealer('Mr.','David','Soliman')

print("-------------------")
print("DB DEALTERS--------")
print(DBdealers)
print("-------------------")


Inserted record into dealers with id 4 and Name Henry
Inserted record into dealers with id 5 and Name Anny
Inserted record into dealers with id 6 and Name David
-------------------
DB DEALTERS--------
{'Henry': 4, 'Anny': 5, 'David': 6}
-------------------


In [14]:
#now, let's insert all records from the dataset.

#let's define a record structure with a class.

class Record:
    
    def __init__(self, CarDescription, Year, Price, KmsDriven, Fuel, SellerType, Transmission, Owner, Mileage, Engine, MaxPower, TorqueNm, TorqueRpmMin, TorqueRpmMax, Seats, Dealer):
        #set data values
        self.Name = CarDescription
        self.Year = Year
        self.Price = Price
        self.KmsDriven = KmsDriven
        self.Fuel = Fuel
        self.SellerType = SellerType
        self.Transmission = Transmission
        self.Owner = Owner
        self.Mileage = Mileage
        self.Engine = Engine
        self.MaxPower = MaxPower
        self.TorqueNm = TorqueNm
        self.TorqueRpmMin = TorqueRpmMin
        self.TorqueRpmMax = TorqueRpmMax
        self.Seats = Seats
        self.Dealer = Dealer
    
        #initialize protected variables
        self._carStatusId = -1
        self._carDealId = -1
        self._carInteriorId = -1
        self._carEngineId = -1
        self._carId = -1
        
         #let's initialize DB objects.
        self._mydb, self._mycursor = self._openDbconn()
        
    
    #destructor
    def __del__(self):
        # closing database connection if open.
        if self._mydb.is_connected():
            self._mycursor.close()
            self._mydb.close()
            

    #open connection and start transaction
    def _openDbconn(self):
        
        mydb = mysql.connector.connect(
                                       host="localhost",
                                       user="root",
                                       password="root",
                                       database="carssystem"
                                       )
        
        autocommit = False #no need for begin transaction
        mycursor = mydb.cursor()
        
        return mydb, mycursor
    
    
    def _PdNanToDbNull(self, s):
        s = s.replace('nan','NULL')
        return s
    
    def _isNull(self, value):
        return value is None or str(value) == '' or np.isnull(value) or str(x) == 'nan' or str(x) == 'NULL'
                      
    def _GetCarId(self):
        Id = None
        #search if it already exists on DB
        sql = "SELECT Id "
        sql +=f"FROM cars "
        sql +=f"WHERE Name = '{self.Name}' "
        self._mycursor.reset()
        self._mycursor.execute(sql)
        myresult = self._mycursor.fetchone()
        #if it does exist
        if not myresult is None:
            Id = myresult[0]
            
        return Id #returns Id (exists) or None (doesn't exist)
    
    
    def _InsertEngine(self):
        #create insert string
        
        tr = DBenums['TransmissionTypes'][self.Transmission] if not pd.isnull(self.Transmission) else 'NULL'
        fl = DBenums['FuelTypes'][self.Fuel] if not pd.isnull(self.Fuel) else 'NULL'
        
        sql = "  INSERT INTO engines "
        sql += " (TransmissionType, PowerCC, MaxPowerBhp, TorqueNm, TorqueRpmMin, TorqueRpmMax, FuelType, MileageKmpl) "
        sql += " VALUES "
        sql +=f" ({tr},{self.Engine},{self.MaxPower},{self.TorqueNm},{self.TorqueRpmMin},{self.TorqueRpmMax},{fl},{self.Mileage}); "
        sql = self._PdNanToDbNull(sql)
        print(sql)
        #execute query and save auto_increment generated id
        self._mycursor.reset()
        self._mycursor.execute(sql)
        self._carEngineId = self._mycursor.lastrowid
        return self._mycursor.lastrowid
    
    
    def _UpdateEngine(self):
        #if there are nulls into the current record
        #and our new record has the useful value for this car model
        #let's update car values
        sql = "SELECT PowerCC, "
        sql +="       MaxPowerBhp, "
        sql +="       TorqueNm, "
        sql +="       TorqueRpmMin, "
        sql +="       TorqueRpmMax, "
        sql +="       MileageKmpl "
        sql +="FROM engines "
        sql +=f"WHERE Id = {self._carEngineId}; "

        self._mycursor.reset()
        self._mycursor.execute(sql)
        myresult = self._mycursor.fetchone()
        #if it does exist
        if not myresult is None:
            #if current record is null and current DB record is valued, then update with old value (ignore current null)
            if self._isNull(self.Engine) and not self._isNull(myresult[0]):
                self.Engine = myresult[0]
            if self._isNull(self.MaxPower) and not self._isNull(myresult[1]):
                self.MaxPower = myresult[1]
            if self._isNull(self.TorqueNm) and not self._isNull(myresult[2]):
                self.TorqueNm = myresult[2]
            if self._isNull(self.TorqueRpmMin) and not self._isNull(myresult[3]):
                self.TorqueRpmMin = myresult[3]
            if self._isNull(self.TorqueRpmMax) and not self._isNull(myresult[4]):
                self.TorqueRpmMax = myresult[4]
            if self._isNull(self.Mileage) and not self._isNull(myresult[5]):
                self.Mileage = myresult[5]
                
            sql = "UPDATE engines "
            sql +="SET "
            sql +=f"PowerCC = {self.Engine}, "
            sql +=f"MaxPowerBhp = {self.MaxPower}, "
            sql +=f"TorqueNm = {self.TorqueNm}, "
            sql +=f"TorqueRpmMin = {self.TorqueRpmMin}, "
            sql +=f"TorqueRpmMax = {self.TorqueRpmMax}, "
            sql +=f"MileageKmpl = {self.Mileage} "
            sql +=f"WHERE Id = {self._carEngineId}; "
            sql = self._PdNanToDbNull(sql)
            #execute query
            self._mycursor.reset()
            self._mycursor.execute(sql)
        
    
    def _InsertInteriors(self):
        #create insert string
        sql = "  INSERT INTO Interiors "
        sql += " (SeatsNumber) "
        sql += " VALUES "
        sql +=f" ({self.Seats}); "
        sql = self._PdNanToDbNull(sql)
        #execute query and save auto_increment generated id
        self._mycursor.reset()
        self._mycursor.execute(sql)
        self._carInteriorId = self._mycursor.lastrowid
        return self._mycursor.lastrowid
    
    def _UpdateInterior(self):
        #if there are nulls into the current record
        #and our new record has the useful value for this car model
        #let's update car values
        #if there are nulls into the current record
        #and our new record has the useful value for this car model
        #let's update car values
        sql = "SELECT SeatsNumber "
        sql +="FROM interiors "
        sql +=f"WHERE Id = {self._carInteriorId}; "

        self._mycursor.reset()
        self._mycursor.execute(sql)
        myresult = self._mycursor.fetchone()
        #if it does exist
        if not myresult is None:
            #if current record is null and current DB record is valued, then update with old value (ignore current null)
            if self._isNull(self.Seats) and not self._isNull(myresult[0]):
                self.Seats = myresult[0]
                
            sql = "UPDATE interiors "
            sql +="SET "
            sql +=f"SeatsNumber = {self.Seats} "
            sql +=f"WHERE Id = {self._carEngineId}; "
            sql = self._PdNanToDbNull(sql)
            #execute query
            self._mycursor.reset()
            self._mycursor.execute(sql)
            
            
    def _InsertCar(self):
        #we insert it on DB
        #create insert string
        sql = "  INSERT INTO cars "
        sql += " (Name, Interior, Engine) "
        sql += " VALUES "
        sql +=f" ('{self.Name}',{self._carInteriorId},{self._carEngineId}); "
        sql = self._PdNanToDbNull(sql)
        #execute query and save auto_increment generated id
        self._mycursor.reset()
        self._mycursor.execute(sql)
        self._carId = self._mycursor.lastrowid
        return self._mycursor.lastrowid
    
            
    def _InsertCarStatus(self):
        #create insert string
        
        ow = DBenums['OwnerTypes'][self.Owner] if not pd.isnull(self.Owner) else 'NULL'
        
        sql = "  INSERT INTO carstatus "
        sql += " (OwnerType, KmsDriven, Year) "
        sql += " VALUES "
        sql +=f" ('{ow}',{self.KmsDriven},'{self.Year}'); "
        sql = self._PdNanToDbNull(sql)
        #execute query and save auto_increment generated id
        self._mycursor.reset()
        self._mycursor.execute(sql)
        self._carStatusId = self._mycursor.lastrowid
        return self._mycursor.lastrowid
    
        
    def _InsertCarDeal(self):
        #create insert string
        
        st = DBenums['SellerTypes'][self.SellerType] if not pd.isnull(self.SellerType) else 'NULL'
        dl = DBdealers[self.Dealer] if not pd.isnull(self.Dealer) else 'NULL'
        
        sql = "  INSERT INTO Deals "
        sql += " (Dealer, SellerType, Price, Car, CarStatus) "
        sql += " VALUES "
        sql +=f" ({dl},{st},{self.Price},{self._carId},{self._carStatusId}); "
        sql = self._PdNanToDbNull(sql)
        #execute query and save auto_increment generated id
        self._mycursor.reset()
        self._mycursor.execute(sql)
        self._carDealId = self._mycursor.lastrowid
        return self._mycursor.lastrowid
        
        
    def Insert(self):
        
        try:
            
            print(f"Try Insert Record for car with Description_ {self.Name}")
            
            if not self._mydb.is_connected(): #if not connected, 
                self._mydb, self._mycursor = self._openDbconn() #open connection and transaction
            
            #let's insert/update car model data
            self._carId = self._GetCarId()
            if self._carId is None: #if the car does not exists
                #let's insert the car
                self._carEngineId = self._InsertEngine()
                self._carInteriorId = self._InsertInteriors()
                self._carId = self._InsertCar()
            else:
                #if there are nulls into the current record
                #and our new record has the useful value for this car model
                #let's update car values
                self._UpdateEngine()
                self._UpdateInterior()
            
            #let's insert specific transaction data
            self._carStatusId = self._InsertCarStatus()
            self._carDealId = self._InsertCarDeal()
            
            #let's commit changes
            self._mydb.commit() 
            print(f"Inserted Record for car with Id {self._carId} and dealId {self._carDealId}")
            
        except ValueError: #in case of error
            self._mydb.rollback() #let's rollback
            print(ValueError) #let's print it
        
        finally:
            # closing database connection.
            if self._mydb.is_connected():
                self._mycursor.close()
                self._mydb.close()
        

In [15]:
for row_index in data.index:
    record = Record(CarDescription=data['name'][row_index], \
                    Year=data['year'][row_index], \
                    Price=data['selling_price'][row_index], \
                    KmsDriven=data['km_driven'][row_index], \
                    Fuel=data['fuel'][row_index], \
                    SellerType=data['seller_type'][row_index], \
                    Transmission=data['transmission'][row_index], \
                    Owner=data['owner'][row_index], \
                    Mileage=data['mileage'][row_index], \
                    Engine=data['engine'][row_index], \
                    MaxPower=data['max_power'][row_index], \
                    TorqueNm=data['ToqueNm'][row_index], \
                    TorqueRpmMin=data['TorqueRpmMin'][row_index], \
                    TorqueRpmMax=data['TorqueRpmMax'][row_index], \
                    Seats=data['seats'][row_index], \
                    Dealer=data['Dealer'][row_index])
    record.Insert()

Try Insert Record for car with Description_ Maruti Swift Dzire VDI
  INSERT INTO engines  (TransmissionType, PowerCC, MaxPowerBhp, TorqueNm, TorqueRpmMin, TorqueRpmMax, FuelType, MileageKmpl)  VALUES  (3,1248.0,74.0,190,2000,NULL,5,23.4); 
Inserted Record for car with Id 8129 and dealId 8129
Try Insert Record for car with Description_ Skoda Rapid 1.5 TDI Ambition
  INSERT INTO engines  (TransmissionType, PowerCC, MaxPowerBhp, TorqueNm, TorqueRpmMin, TorqueRpmMax, FuelType, MileageKmpl)  VALUES  (3,1498.0,103.52,250,1500,2500,5,21.14); 
Inserted Record for car with Id 8130 and dealId 8130
Try Insert Record for car with Description_ Honda City 2017-2020 EXi
  INSERT INTO engines  (TransmissionType, PowerCC, MaxPowerBhp, TorqueNm, TorqueRpmMin, TorqueRpmMax, FuelType, MileageKmpl)  VALUES  (3,1497.0,78.0,124.54445499999999,2.700,NULL,6,17.7); 
Inserted Record for car with Id 8131 and dealId 8131
Try Insert Record for car with Description_ Hyundai i20 Sportz Diesel
  INSERT INTO engines  (